# Run the final state algorithm on a ROOT HSDATA file

First create an object of your final state class. You may configure it if you want to limit the number of particles in an event or set to just run on generated simulation data.
This notebook is equivalent to the RunFSRootXXX.C macro which will be in the code directory.
Again just change XXX for your class name.
The string arguements set which particles you would like to use their pre-determined PID for and which you would like to be inclusive (i.e. any number of thse particles).
**Example**, 
1. for tagged photon experiment I might want fs("Beam","Beam")
This means use predetermined photon ID (i.e. I know it came from the tagger) and I will analyse events with any number of tagger photons.
2. fs("e-","proton:pi+:pi-")  i.e. use predetemined electron PID and just determine hadron species based on charge and try all combinations in events with any number of proton,pi+,pi-, but with the number of electrons defined for the topology
3. fs("NONE","ALL") => do not use any PID info other than charge, analyse events with any number of particles (this can be limited by SetMaxParticles)

In [1]:
gROOT->ProcessLine(".x LoadPi2.C+"); //Load the classes

%%%%%%%%%%%%%%%%%%%%%%%%%    BaseEventInfo
%%%%%%%%%%%%%%%%%%%%%%%%%    BaseRunInfo
%%%%%%%%%%%%%%%%%%%%%%%%%    THSWeights
%%%%%%%%%%%%%%%%%%%%%%%%%    DataManager
%%%%%%%%%%%%%%%%%%%%%%%%%    LundReader
12121212121212121212121212 EventInfo
12121212121212121212121212 RunInfo
-I$ROOTSYS/include -D__LZ4__ -I/work/clas12/Clas12Tool//Lz4/lib -I"/work/root/v6.14.04/etc" -I"/work/root/v6.14.04/etc/cling" -I"/work/root/v6.14.04/include" -I"/usr/include/python2.7" -I"/work/Dropbox/HaSpect/dev/HASPECT6/HaSpect/" -I"/work/Dropbox/HaSpect/dev/HASPECT6/hsclas12/" -I"/work/Dropbox/HaSpect/dev/HASPECT6/hsexperiments/clas12/" -I"/usr/include/freetype2" -I"/work/Dropbox/HaSpect/dev/HASPECT6/hsdata/" -I"/work/Dropbox/HaSpect/dev/HASPECT6/"
HHHHHHHHHHHHHHHHHHHHHHHH    HipoReader
HHHHHHHHHHHHHHHHHHHHHHHH    HipoTrigger
HHHHHHHHHHHHHHHHHHHHHHHH    HipoDST
&&&&&&&&&&&&&&&&&&&&&&&&&&&& HSKinematics
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Cuts
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Combitorial
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Parti

In [2]:
 Pi2 fs("NONE","ALL"); //string arguments => PID, INCLUSIVE
  // fs.SetGenerated(); //just analyse generated branch
  // fs.SetMaxParticles(10); //max number of particles of any 1 type
 

 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
 Topology::Print() : 0
    particles = -10000 -10000 10000 10000 
You can have any number of the following particles : 
 ALL 
The following particle are identified by pdg code : 
 ALL 
 ParticleIter::Print() 
     Type : 2 number chosen 1 of id  11 and number used here = 1
 ParticleIter::Print() 
     Type : 2 number chosen 1 of id  2212 and number used here = 1
 ParticleIter::Print() 
     Type : 2 number chosen 1 o

Create an output tree. Here we will use a FiledTree object which is a wrapper class to a tree in a file. We will also create this as a smart pointer so we can delete it at the end by calling reset(), this will save the tree to disk. The Filed tree needs 2 strings ,1 a treename and the other the file name. You should give the full path for where you would like the files saved. It will overwrite any existing files. 

In [3]:
  //strings = treename and filename (give full path)
  auto output = new FiledTree("FinalTree","output.root");
  fs.FinalStateOutTree(output->Tree()); //connect ouput tree to final state branches

Here I use a predefined CLAS12DeltaTime cuts class for PID. It takes 4 arguments which are cuts centred on zero for FT, FD, CD, FDCAL (for neutra) respectively.

I define specific cuts for proton and electron, while restrict other particles (here just pions) to only be in CD.

In [4]:
ParticleCutsManager cutsman;
auto eCut=make_shared<CLAS12DeltaTime>(4,4,0,0); //4ns FT,FD
auto protCut=make_shared<CLAS12DeltaTime>(0,5,5,0); //5ns FD,CD
auto otherCut=make_shared<CLAS12DeltaTime>(0,5,0,0); //5ns FD
cutsman.AddParticleCut("e-",eCut); //assign to manager
cutsman.AddParticleCut("proton",protCut); //assign to manager
cutsman.SetDefaultCut(otherCut); //assign to manager
cutsman.ConfigureCuts(&fs);   
fs.RegisterPostTopoAction(&cutsman);

Info ParticleCutsManager AddParticleCut set cut for 0
Info ParticleCutsManager AddParticleCut set cut for 11
Info ParticleCutsManager AddParticleCut set cut for 2212


Now we configure the input data. Here we use a ROOT file presaved in HSDATA format. It is possible to analyse other dataformats via the DataManager class.

In [5]:
 //create datamanager
  auto dm=std::make_shared<DataManager>();

  //And make a chain of data files
  TChain chain("HSParticles");
  chain.Add("/work/dump/hsout/out_clas_pin_004909.evio.00228.hipo.root");
  dm->InitChain(&chain);

  //connect FinalState to Data by moving the pointer
  fs.SetDataManager(dm);

Info in <DataManager::InitChain>:  Will proceess all 1 files in chain


DataManager::InitReader set branch 0x7f0c61a622d0 0x7f0c61a622d0
DataManager::InitReader EventInfo
DataManager::InitReader RunInfo


Analyse all the events. Not you may prefer just looping over the data manager directly via while(dm->ReadEvent()) fs->Process();
You can also supply a number of events to analyse e.g. fs.ProcessData(100);

In [6]:
cout<<"Number of Events to process "<<chain.GetEntries()<<endl;
fs.ProcessData(); //No number give, analyse all events in chain

Number of Events to process 51573
51573 1 1
DataManager::NextChainFile() Finished all files 
******TreeCache statistics for tree: HSParticles in file: /work/dump/hsout/out_clas_pin_004909.evio.00228.hipo.root ******
TreeCache = 50 MBytes
N leaves  = 30
ReadTotal = 8.67032 MBytes
ReadUnZip = 24.5835 MBytes
ReadCalls = 130
ReadSize  =  66.695 KBytes/read
Readahead = 256 KBytes
Readextra =  0.00 per cent
Real Time =   0.588 seconds
CPU  Time =   0.530 seconds
Disk Time =   0.003 seconds
Disk IO   = 2870.694 MBytes/s
ReadUZRT  =  41.776 MBytes/s
ReadUZCP  =  46.384 MBytes/s
ReadRT    =  14.734 MBytes/s
ReadCP    =  16.359 MBytes/s


In [7]:
%jsroot

Draw some variables from your tree.

In [8]:
TCanvas c1("c1","c1");
output->Tree()->Draw("MissMass>>(100,-1,2)","Topo==1");
output->Tree()->Draw("MissMass","Topo==0","same");
output->Tree()->Draw("MissMass","Topo==2","same");
output->Tree()->Draw("MissMass","Topo==3","same");
c1.Draw();

And remember to save the tree!

In [9]:
delete output;